In [241]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [242]:
import hit_rates as hr
import rankfmlib as fmlib
# new library for performance studies built from rankfmlib to read a single file rather than yearly files.
import newlib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from rankfm.rankfm import RankFM
import function_lib as flib

from rankfm.evaluation import hit_rate

Read member attributes

In [243]:
%%time
# Perhaps add an argument stating which colums are attributes
interaction_dct = newlib.read_data_attributes_single_file("activity_reduced_with_attributes.csv",
                                                        age_cuts=[0,30,50,70,150])

CPU times: user 33.1 ms, sys: 31 µs, total: 33.1 ms
Wall time: 33.1 ms


In [244]:
%%time
newlib.train_valid_dct(interaction_dct, train_perc=0.7, valid_perc=0.1, temporal=False)
print(interaction_dct.keys())

(714086, 2)
dict_keys(['df_members', 'df_user_attr', 'df_item_attr', 'data_train', 'data_valid', 'data_test'])
CPU times: user 80.7 ms, sys: 8 µs, total: 80.7 ms
Wall time: 80.8 ms


In [245]:
data_train = interaction_dct['data_train']
data_valid = interaction_dct['data_valid']
df_members = interaction_dct['df_members']
df_user_attrib = interaction_dct['df_user_attr']
data_train.shape, data_valid.shape, df_members.shape, 

((180388, 2), (49069, 2), (714086, 2))

In [246]:
# Check that member_attrib is a superset of data_train
df_member_attrib = interaction_dct['df_user_attr']
set(data_train.MEMBER_ID) - set(df_member_attrib.MEMBER_ID)

set()

In [247]:
loss = 'warp'
loss = 'bpr'  # nb neg samples = 1
model = RankFM(factors=20, loss=loss, max_samples=10000, alpha=0.05, beta=0.1, learning_rate=0.1, learning_schedule='constant')

The problem to solve: Why are results worse with filter=False? To figure this out, I may have to write my own recommender and hit_rate functions. 
 1) optimize parameter without attributes
 2) optimize age brackets using just one attribute
 3) optimize temperature brackets (destination features) using just one attribute
 4) add gender, country of origin to the member attributes
 
 One problem found was that I was removing duplicate Member/Destinations too early. They must be removed from 
 the training and validation sets separately. Now I get up to 40% if data is not filtered. However, I only get 10% accuracy
 when the previous data is filtered. WHY? 

___

Temporal=True

In [248]:
nb_epochs = [50, 100, 250, 500]
#
for epochs in nb_epochs:
    print("-- nb epochs: ", epochs)
    newlib.run_model(model, interaction_dct, nb_epochs=epochs, with_attrib=False)
    print("------------------------------------------")

-- nb epochs:  50
hr (previous filtered):  0.09409471837848372
hr (previous not filtered):  0.3880335217306568
------------------------------------------
-- nb epochs:  100


KeyboardInterrupt: 

In [17]:
nb_epochs = [50, 100, 250, 500]
#
for epochs in nb_epochs:
    print("-- nb epochs: ", epochs)
    newlib.run_model(model, interaction_dct, nb_epochs=epochs, with_attrib=True)
    print("------------------------------------------")

-- nb epochs:  50
dict_keys(['df_members', 'df_user_attr', 'df_item_attr', 'data_train', 'data_valid', 'data_test'])
hr (previous filtered):  0.08633373554941419
hr (previous not filtered):  0.35798528667626794
------------------------------------------
-- nb epochs:  100
dict_keys(['df_members', 'df_user_attr', 'df_item_attr', 'data_train', 'data_valid', 'data_test'])
hr (previous filtered):  0.09723249386944845
hr (previous not filtered):  0.38663345140321514
------------------------------------------
-- nb epochs:  250
dict_keys(['df_members', 'df_user_attr', 'df_item_attr', 'data_train', 'data_valid', 'data_test'])
hr (previous filtered):  0.09260052158343389
hr (previous not filtered):  0.4506247323965591
------------------------------------------
-- nb epochs:  500
dict_keys(['df_members', 'df_user_attr', 'df_item_attr', 'data_train', 'data_valid', 'data_test'])
hr (previous filtered):  0.1001907282706006
hr (previous not filtered):  0.46545482853917713
--------------------------

### Temporal=False
- The results seem independent of whether the flights in the validation data come 
after the flights in the training set. 
- NOT CLEAR WHY. So I still have not 
resolved the issue of the hit rates. 
- I must now create my own functions to evaluate the hits to better understand. 

In [14]:
nb_epochs = [50, 100, 250, 500, 1000]
nb_epochs = [50, 100, 250]
#
for epochs in nb_epochs:
    print("-- nb epochs: ", epochs)
    newlib.run_model(model, interaction_dct, nb_epochs=epochs)
    print("------------------------------------------")

-- nb epochs:  50
dict_keys(['df_members', 'df_user_attr', 'df_item_attr', 'data_train', 'data_valid', 'data_test'])
hr (previous filtered):  0.08606126659141333
hr (previous not filtered):  0.3522245144213927
------------------------------------------
-- nb epochs:  100
dict_keys(['df_members', 'df_user_attr', 'df_item_attr', 'data_train', 'data_valid', 'data_test'])
hr (previous filtered):  0.10346035576661088
hr (previous not filtered):  0.39387334085866643
------------------------------------------
-- nb epochs:  250
dict_keys(['df_members', 'df_user_attr', 'df_item_attr', 'data_train', 'data_valid', 'data_test'])


KeyboardInterrupt: 

# Create newlib.recommender(data_train)

In [249]:
interaction_dct.keys()

dict_keys(['df_members', 'df_user_attr', 'df_item_attr', 'data_train', 'data_valid', 'data_test'])

1. create dataframe list list of destinations for each member. Do this for training and validation

In [338]:
newlib.run_model(model, interaction_dct, nb_epochs=500, topN=5, with_attrib=False)
#topN=5, nb_epochs=500, hr=0.16 (filtered), hr=0.64 (not filtered), with_attrib=True`

hr (previous filtered):  0.15911128435002925
hr (previous not filtered):  0.6595985188072501


In [337]:
%%time
# no filter: 63% accurate. That makes sense. 
# with filter of hits in training set: 11% (same results as those of rankfm)
# Same answers when repeated multiple times
result = newlib.recommender(model, interaction_dct, keep_nb_members=None, topN=5)
# topN=5, nb_epochs=500, hr=0.11 (filtered), hr=0.6 (not filtered), with_attrib=True

hit rate (without previous filter) =  0.61770138339176
hit rate (with previous filter) =  0.15887741181056325
CPU times: user 12.5 s, sys: 83.6 ms, total: 12.6 s
Wall time: 12.5 s


In [332]:
%%time
# no filter: 63% accurate. That makes sense. 
# with filter of hits in training set: 11% (same results as those of rankfm)
result = newlib.recommender(model, interaction_dct, keep_nb_members=None, topN=5)

hit rate (without previous filter) =  0.61770138339176
hit rate (with previous filter) =  0.17442993568505163
CPU times: user 13.1 s, sys: 50.7 ms, total: 13.1 s
Wall time: 13.1 s


In [ ]:
pd.DataFrame.reset_index()

In [193]:
result.head()

,MEMBER_ID,D,pred
0,239300926,ADZ,-0.401
1,239300926,ASU,0.062
2,239300926,ATL,-1.229
3,239300926,AUA,-0.033
4,239300926,BAQ,-0.175


In [156]:
res1 = result.groupby('MEMBER_ID').agg({'D':list, 'pred':list})
res1.head(5)

,D,pred
MEMBER_ID,,
100031892,"[ADZ, ASU, ATL, AUA, BAQ, BGA, BGI, BOG, BOS, ...","[-0.7474620938301086, -0.43247297406196594, -1..."
100034364,"[ADZ, ASU, ATL, AUA, BAQ, BGA, BGI, BOG, BOS, ...","[-0.6220289468765259, -0.6381959915161133, -1...."
100034902,"[ADZ, ASU, ATL, AUA, BAQ, BGA, BGI, BOG, BOS, ...","[-0.885479211807251, -0.07757009565830231, -1...."
100035145,"[ADZ, ASU, ATL, AUA, BAQ, BGA, BGI, BOG, BOS, ...","[-0.7716412544250488, -0.4266751706600189, -1...."
100040465,"[ADZ, ASU, ATL, AUA, BAQ, BGA, BGI, BOG, BOS, ...","[-0.8271920084953308, 0.0317193977534771, -1.4..."


In [157]:
# Sort pred numerically using argsort, and sort D in the same order. 
res1.apply(lambda x: len(x)), res1.shape

(D       26459
 pred    26459
 dtype: int64,
 (26459, 2))

In [158]:
%%time 
res1['argsort'] = res1['pred'].map(np.argsort)
res1.head(5)

CPU times: user 173 ms, sys: 0 ns, total: 173 ms
Wall time: 173 ms


,D,pred,argsort
MEMBER_ID,,,
100031892,"[ADZ, ASU, ATL, AUA, BAQ, BGA, BGI, BOG, BOS, ...","[-0.7474620938301086, -0.43247297406196594, -1...","[80, 2, 74, 12, 73, 72, 56, 82, 24, 32, 20, 63..."
100034364,"[ADZ, ASU, ATL, AUA, BAQ, BGA, BGI, BOG, BOS, ...","[-0.6220289468765259, -0.6381959915161133, -1....","[74, 80, 12, 2, 72, 24, 82, 56, 73, 32, 63, 45..."
100034902,"[ADZ, ASU, ATL, AUA, BAQ, BGA, BGI, BOG, BOS, ...","[-0.885479211807251, -0.07757009565830231, -1....","[2, 74, 82, 72, 80, 85, 45, 12, 24, 55, 56, 17..."
100035145,"[ADZ, ASU, ATL, AUA, BAQ, BGA, BGI, BOG, BOS, ...","[-0.7716412544250488, -0.4266751706600189, -1....","[80, 2, 74, 73, 12, 72, 82, 56, 32, 24, 20, 63..."
100040465,"[ADZ, ASU, ATL, AUA, BAQ, BGA, BGI, BOG, BOS, ...","[-0.8271920084953308, 0.0317193977534771, -1.4...","[2, 72, 82, 74, 80, 56, 73, 12, 55, 32, 24, 85..."


In [174]:
%%time 
topN = 5
argsort = res1['pred'].apply(lambda x: np.argsort(x)[::-1])
res1['argsort'] = argsort

def argsortcolD(row):
    return np.array(row['D'])[row['argsort'][0:topN]]

def argsortcolPred(row):
    return np.array(row['pred'])[row['argsort'][0:topN]]

res1['D1'] = res1.apply(argsortcolD, axis=1)
res1['pred1'] = res1.apply(argsortcolPred, axis=1)
res2 = res1.drop(['D','pred', 'argsort'], axis=1)
res2.head(50)

CPU times: user 911 ms, sys: 12.1 ms, total: 923 ms
Wall time: 909 ms


,D1,pred1
MEMBER_ID,,
100031892,"[MEX, PTY, GUA, MIA, SJO]","[1.473193883895874, 1.3925684690475464, 1.3717..."
100034364,"[PTY, MIA, MEX, SJO, BOG]","[1.3868271112442017, 1.298481822013855, 1.2900..."
100034902,"[PTY, SJO, MEX, GUA, SDQ]","[1.8354800939559937, 1.4655382633209229, 1.460..."
100035145,"[MEX, PTY, GUA, SJO, MIA]","[1.537152647972107, 1.4331413507461548, 1.4235..."
100040465,"[PTY, MEX, SJO, GUA, LIM]","[1.4250928163528442, 1.2827187776565552, 1.156..."
100041338,"[PTY, MIA, CUN, MEX, SJO]","[1.2134510278701782, 1.1528159379959106, 1.077..."
100041994,"[MCO, MIA, GUA, LAX, SJO]","[1.1942585706710815, 1.1734713315963745, 1.094..."
100043486,"[MIA, MEX, PTY, BOG, GUA]","[1.3955706357955933, 1.3718065023422241, 1.360..."
100043615,"[PTY, MEX, SJO, GUA, SDQ]","[1.84485924243927, 1.5330605506896973, 1.51278..."


In [169]:
# number of flights in validation set with PTY, SJO as destination
x = interaction_dct['data_valid']
x[x['D'] == 'SJO'].shape, x[x['D'] == 'PTY'].shape

((2902, 2), (4957, 2))

In [171]:
# number of flights in validation set with PTY, SJO as destination
x = interaction_dct['data_test']
x[x['D'] == 'SJO'].shape, x[x['D'] == 'PTY'].shape, x.shape

((4486, 2), (7038, 2), (81326, 2))

In [172]:
x = interaction_dct['data_train']
x[x['D'] == 'SJO'].shape, x[x['D'] == 'PTY'].shape, x.shape

((8335, 2), (11409, 2), (180261, 2))

In [173]:
x = interaction_dct['df_members']
x[x['D'] == 'SJO'].shape, x[x['D'] == 'PTY'].shape, x.shape
# 90k PTY and 48k SJO destinations out of 714,000 member flights

((48355, 2), (90101, 2), (714086, 2))

In [ ]:
df['type'] = df['type'].map(np.sort).map(list) 

In [99]:
newlib.recommender(model, interaction_dct)

nb destinations:  (86,)
members:  [239300926 230131088 232922338 ... 254785475 233485864 248525058]
completed pair construction
pairs shape:  (2275474, 2)
(2275474, 3)
         MEMBER_ID    D   pred
0        239300926  ADZ -0.401
1        239300926  ASU  0.062
2        239300926  ATL -1.229
3        239300926  AUA -0.033
4        239300926  BAQ -0.175
...            ...  ...    ...
2275469  248525058  VLN -0.867
2275470  248525058  VSA -1.625
2275471  248525058  VVI -0.683
2275472  248525058  YUL -1.367
2275473  248525058  YYZ -1.298

[2275474 rows x 3 columns]


In [94]:
interaction_dct['data_valid']['MEMBER_ID'].unique().shape[0]*86

2275474

In [18]:
interaction_dct.keys()

dict_keys(['df_members', 'df_user_attr', 'df_item_attr', 'data_train', 'data_valid', 'data_test', 'train_dest_sets', 'valid_dest_sets'])

In [ ]:
model.predict()

In [ ]:
np.argsort()

In [70]:
a = [1,2,3,4,5]
a = np.array(a)
a = a[::-1]
a

array([5, 4, 3, 2, 1])

In [ ]:
data_valid = interaction_dct['data_valid']
model.predict(data_valid, cold_start='drop').shape

In [ ]:
valid.data.shape

In [ ]:
inf = 'activity_reduced_with_attributes.csv'
dff = pd.read_csv(inf)
dff.groupby(['MEMBER_ID','D']).size().max()
# result is 1. So I removed too much data, since a M-D combination occuring in the training 
# set cannot possibly occur in the validation set. 

In [ ]:
interaction_dct.keys()

When calling model.recommend, whether with filter_previous True or False, the first recommendation is mostly SJO and PTY. That does not seem correct. 
Note that the hit rate is having issues.

In [ ]:
model.recommend(data_train.MEMBER_ID, filter_previous=True, n_items=5).head(5)

In [ ]:
interaction_dct['data_train']

In [ ]:
flib.hit_rate(model, interaction_dct['data_valid'])

In [ ]:
members_train = data_train.MEMBER_ID.unique()  # returns ndarray
members_valid = data_valid.MEMBER_ID.unique()

In [ ]:
mtrain = set(members_train)
mvalid = set(members_valid)
len(mvalid), len(mtrain), mvalid.intersection(mtrain);

In [ ]:
model.recommend(members_valid, n_items=5, filter_previous=True, cold_start='drop')

In [ ]:
members_valid.shape

In [ ]:
mvalid_np = np.asarray(mvalid)

In [ ]:
data_valid

In [ ]:
data_test

In [ ]:
hit_rate(model, data_test, k=5, filter_previous=True)

In [ ]:
# More iterations, worse results. Why would that be? Unless iterations is not converged?
interactions_dct, model = newlib.calculate_dct_with_attributes(nb_samples=1000, with_attrib=True, verbose=False, nb_epochs=100, loss='warp')

In [ ]:
# Hit rate goes fro 0.29 to 0.21 when nb_epochs goes from 30 to 100. I DO NOT LIKE THIS. 
newlib.topn_recommendations_with_attributes(model, interactions_dct, topN=3)

In [ ]:
Construct training/testing datasets
# Consider a training set of 10% of the data. 
# Shuffle the data first


In [ ]:
data = interactions_dct['train']
data1 = data.sample(frac=1, random_state=100)

In [ ]:
data.head(), data1.head()

In [ ]:
data.groupby('MEMBER_ID').agg({'D':list}).shape

In [ ]:
print(x[])

In [ ]:
hit_rate(model, data, k=5, filter_previous=False) # cold_start='drop');

Calculate the hit rate myself based on the recommendations, which are based on the ranking scores. 
To do this, 
* first create a list of destinations for each member in the training set. 
* next, for each of the recommendations, determine whether the member flew there in the test set. 
* if the member flew there in the test set, add 1 to the hit list, else 0

In [ ]:
data = interactions_dct['train']
x = list(interactions_dct['train']['MEMBER_ID'])
data_train, data_valid, data_test = newlib.train_valid(x, 0.8, 0.1)

Train the model

In [ ]:
in_file = "activity_reduced_with_attributes.csv"
member_dest_df, df_user_attrib, df_item_attrib = newlib.read_data_attributes_single_file(in_file)